# Задание 3

In [7]:
import numpy as np

In [8]:
seq1 = "TGAGGAC"
seq2 = "TAGGACT"

## Алгоритм Нидлмана-Вунша

In [9]:
def needleman_wunsch(seq1, seq2, match_score=2, mismatch_penalty=-1, gap_penalty=-2):
    n = len(seq1)
    m = len(seq2)
    
    #Создаём матрицу
    score_matrix = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        score_matrix[i][0] = i * gap_penalty
    for j in range(1, m+1):
        score_matrix[0][j] = j * gap_penalty
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)
    
    #Выравнивание
    align1, align2 = '', ''
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and j > 0 and score_matrix[i][j] == score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty):
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif i > 0 and score_matrix[i][j] == score_matrix[i-1][j] + gap_penalty:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1
    
    return align1, align2, score_matrix[n][m]

In [10]:
nw_align1, nw_align2, nw_score = needleman_wunsch(seq1, seq2)
print("Needleman-Wunsch Alignment:")
print(nw_align1)
print(nw_align2)
print("Score:", nw_score)

Needleman-Wunsch Alignment:
TGAGGAC-
T-AGGACT
Score: 8.0


## Алгоритм Смита-Вотермана

In [11]:
def smith_waterman(seq1, seq2, match_score=2, mismatch_penalty=-1, gap_penalty=-2):
    n = len(seq1)
    m = len(seq2)
    
    #Создаём матрицу
    score_matrix = np.zeros((n+1, m+1))
    max_score = 0
    max_pos = (0, 0)
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(0, match, delete, insert)
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_pos = (i, j)
    
    #Выпавнивание
    align1, align2 = '', ''
    i, j = max_pos
    while score_matrix[i][j] != 0:
        if score_matrix[i][j] == score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty):
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif score_matrix[i][j] == score_matrix[i-1][j] + gap_penalty:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1
    
    return align1, align2, max_score

In [12]:
sw_align1, sw_align2, sw_score = smith_waterman(seq1, seq2)
print("\nSmith-Waterman Alignment:")
print(sw_align1)
print(sw_align2)
print("Score:", sw_score)


Smith-Waterman Alignment:
AGGAC
AGGAC
Score: 10.0


## Глобальное выравнивание через Python

In [13]:
from Bio.Align import substitution_matrices
from Bio.Align import PairwiseAligner

In [14]:
aligner = PairwiseAligner()

aligner.mode = 'global'
aligner.match_score = 2  #Совпадение
aligner.mismatch_score = -1  #Несовпадение
aligner.open_gap_score = -2  # Штраф за gap
aligner.extend_gap_score = -2  # Штраф за gap

seq1 = "TGAGGAC"
seq2 = "TAGGACT"

alignments = aligner.align(seq1, seq2)
for alignment in alignments:
    print(alignment)
    print("Счет:", alignment.score)

target            0 TGAGGAC- 7
                  0 |-|||||- 8
query             0 T-AGGACT 7

Счет: 8.0
